![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_ONNX_in_Spark_NLP_RoBertaSentenceEmbeddings.ipynb)

## Import RoBertaSentenceEmbeddings models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for RoBertaSentenceEmbeddings from HuggingFace but they have to be compatible with `Onnx` and they have to be in `Fill Mask` category. Meaning, you cannot use RoBertaSentenceEmbeddings models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.4`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.4 optimum onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [FacebookAI/roberta-base](https://huggingface.co/FacebookAI/roberta-base) model from HuggingFace as an example
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import RobertaTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "FacebookAI/roberta-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('onnx_models/FacebookAI/roberta-base/tokenizer_config.json',
 'onnx_models/FacebookAI/roberta-base/special_tokens_map.json',
 'onnx_models/FacebookAI/roberta-base/vocab.json',
 'onnx_models/FacebookAI/roberta-base/merges.txt',
 'onnx_models/FacebookAI/roberta-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l onnx_models/{MODEL_NAME}

total 486304
-rw-r--r-- 1 root root       609 Jun 16 20:58 config.json
-rw-r--r-- 1 root root    456318 Jun 16 20:58 merges.txt
-rw-r--r-- 1 root root 496500406 Jun 16 20:58 model.onnx
-rw-r--r-- 1 root root       958 Jun 16 20:58 special_tokens_map.json
-rw-r--r-- 1 root root      1222 Jun 16 20:58 tokenizer_config.json
-rw-r--r-- 1 root root    999355 Jun 16 20:58 vocab.json


- We need to move `vocab.txt` and `merges.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find them. However, before doing that, we first need to convert Hugging Face’s `vocab.json` into a plain `vocab.txt` format, as Spark NLP does not support the JSON format.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [4]:
import json

!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

with open(f"{EXPORT_PATH}/vocab.json") as f, open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as out:
    out.write("\n".join(json.load(f)))

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(ort_model.config.id2label[k] for k in sorted(ort_model.config.id2label)))

## Import and Save RoBertaSentenceEmbeddings in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 20.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `RoBertaSentenceEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `RoBertaSentenceEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [7]:
from sparknlp.annotator import RoBertaSentenceEmbeddings

roberta = RoBertaSentenceEmbeddings.loadSavedModel(f"onnx_models/{MODEL_NAME}", spark)\
    .setInputCols(["sentence"])\
    .setOutputCol("roberta")\
    .setCaseSensitive(True)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [8]:
roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX RoBertaSentenceEmbeddings model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {MODEL_NAME}_spark_nlp

total 484952
drwxr-xr-x 4 root root      4096 Jun 16 21:04 fields
drwxr-xr-x 2 root root      4096 Jun 16 21:04 metadata
-rw-r--r-- 1 root root 496576300 Jun 16 21:04 roberta_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaSentenceEmbeddings model 😊

In [11]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetectorDLModel
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

embeddings = RoBertaSentenceEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["sentence"]) \
    .setOutputCol("roberta")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    embeddings
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(roberta.embeddings[0]) as embeddings").show()

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
+-------------+
|   embeddings|
+-------------+
|-0.0064373845|
| 1.1743068E-4|
| 0.0012736885|
| -0.004747964|
| 0.0071578906|
| -0.012505115|
| -0.004461373|
|  0.004919375|
| 0.0075282366|
| 8.4523397E-4|
|-0.0043041506|
|  0.011945481|
| -0.002451635|
|-0.0014557999|
|  0.006214544|
|-0.0051567056|
|  -0.00794371|
|-0.0021585438|
| 0.0060420455|
|-0.0017529879|
+-------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of RoBertaSentenceEmbeddings models from HuggingFace 🤗 in Spark NLP 🚀
